<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/cpp_functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

In [ ]:
# 手続き型の例
# Listing 1.1 Calculating the number of lines the imperative way
%%script false
vector<int> count_lines_in_files(const vector<string>& files)
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}